## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Henties Bay,NaN,-22.1160,14.2845,69.01,79,4,10.00,clear sky
1,1,Fairbanks,US,64.8378,-147.7164,33.80,100,90,5.75,light rain
2,2,Ulaangom,MN,49.9811,92.0667,32.36,43,0,2.04,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,44.60,65,20,8.05,few clouds
4,4,Hithadhoo,MV,-0.6000,73.0833,83.82,74,75,17.34,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 20
What is the maximum temperature you would like for your trip? 55


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
spring_skiing_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
spring_skiing_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Fairbanks,US,64.8378,-147.7164,33.80,100,90,5.75,light rain
2,2,Ulaangom,MN,49.9811,92.0667,32.36,43,0,2.04,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,44.60,65,20,8.05,few clouds
5,5,Saravan,AM,39.7196,45.6483,51.80,46,40,4.61,scattered clouds
9,9,Albany,US,42.6001,-73.9662,54.00,20,30,1.01,scattered clouds
17,17,Jamestown,US,42.0970,-79.2353,53.01,40,1,8.05,clear sky
19,19,Kamenka,RU,51.3223,42.7678,39.99,81,100,5.86,overcast clouds
20,20,Kodiak,US,57.7900,-152.4072,33.80,80,1,4.61,clear sky
24,24,Portland,US,45.5234,-122.6762,52.00,66,75,10.36,broken clouds
33,33,Xining,CN,36.6167,101.7667,37.33,64,98,1.86,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
spring_skiing_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows. No null values but just for practice.
if spring_skiing_df.isnull().values.any() == True:
    spring_skiing_df = spring_skiing_df.dropna()
clean_df = spring_skiing_df

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Fairbanks,US,33.80,light rain,64.8378,-147.7164,
2,Ulaangom,MN,32.36,clear sky,49.9811,92.0667,
3,Ushuaia,AR,44.60,few clouds,-54.8000,-68.3000,
5,Saravan,AM,51.80,scattered clouds,39.7196,45.6483,
9,Albany,US,54.00,scattered clouds,42.6001,-73.9662,
17,Jamestown,US,53.01,clear sky,42.0970,-79.2353,
19,Kamenka,RU,39.99,overcast clouds,51.3223,42.7678,
20,Kodiak,US,33.80,clear sky,57.7900,-152.4072,
24,Portland,US,52.00,broken clouds,45.5234,-122.6762,
33,Xining,CN,37.33,overcast clouds,36.6167,101.7667,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.{index}")

Hotel not found... skipping.{index}
Hotel not found... skipping.{index}
Hotel not found... skipping.{index}
Hotel not found... skipping.{index}
Hotel not found... skipping.{index}
Hotel not found... skipping.{index}
Hotel not found... skipping.{index}
Hotel not found... skipping.{index}
Hotel not found... skipping.{index}
Hotel not found... skipping.{index}
Hotel not found... skipping.{index}
Hotel not found... skipping.{index}
Hotel not found... skipping.{index}
Hotel not found... skipping.{index}
Hotel not found... skipping.{index}
Hotel not found... skipping.{index}
Hotel not found... skipping.{index}
Hotel not found... skipping.{index}
Hotel not found... skipping.{index}
Hotel not found... skipping.{index}
Hotel not found... skipping.{index}
Hotel not found... skipping.{index}
Hotel not found... skipping.{index}
Hotel not found... skipping.{index}
Hotel not found... skipping.{index}
Hotel not found... skipping.{index}


In [9]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Fairbanks,US,33.80,light rain,64.8378,-147.7164,Westmark Fairbanks Hotel & Conference Center
2,Ulaangom,MN,32.36,clear sky,49.9811,92.0667,Achit Nuur Hotel & Restaurant
3,Ushuaia,AR,44.60,few clouds,-54.8000,-68.3000,Albatros Hotel
17,Jamestown,US,53.01,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
20,Kodiak,US,33.80,clear sky,57.7900,-152.4072,Best Western Kodiak Inn And Convention Center
24,Portland,US,52.00,broken clouds,45.5234,-122.6762,Embassy Suites by Hilton Portland Downtown
33,Xining,CN,37.33,overcast clouds,36.6167,101.7667,Super Leisure Hotel
41,Pervomaysk,UA,45.66,overcast clouds,48.0443,30.8507,Отель МАЕТОК
43,Pinawa,CA,50.45,overcast clouds,50.1489,-95.8811,Pinawa Motel
47,Tasiilaq,GL,23.00,broken clouds,65.6145,-37.6368,The Red House


In [10]:
clean_hotel_df = hotel_df

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))